In [1]:
import os, re
import nltk, sklearn
import pandas as pd
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup as BS
import json
import numpy as np

In [2]:
blog2013_path = "G:\\author profiling\\pan13-author-profiling-training-corpus-2013-01-09\\en\\"
blog2014_path = "G:\\author profiling\\pan14-author-profiling-training-corpus-english-blogs-2014-04-16\\"
review2014_path = "G:\\author profiling\\pan14-author-profiling-training-corpus-english-reviews-2014-04-16\\"
social2014_path = "G:\\author profiling\\pan14-author-profiling-training-corpus-english-socialmedia-2014-04-16\\"
twitter2015_path = "G:\\author profiling\\pan15-author-profiling-training-dataset-2015-04-23\\pan15-author-profiling-training-dataset-english-2015-04-23\\"
twitter2016_path = "G:\\author profiling\\pan16-author-profiling-training-dataset-2016-04-25\\pan16-author-profiling-training-dataset-english-2016-04-25\\"

In [3]:
#files = os.listdir(file_path)
blog2013_files = os.listdir(blog2013_path)
blog2014_files = os.listdir(blog2014_path)
review2014_files = os.listdir(review2014_path)
social2014_files = os.listdir(social2014_path)
twitter2015_files = os.listdir(twitter2015_path)
twitter2016_files = os.listdir(twitter2016_path)

In [4]:
if len(blog2013_files)>200:
    blog2013_files = np.random.choice(blog2013_files[:-1], 200)
    print "blog2013_files",len(blog2013_files)
else:
    blog2013_files = blog2013_files[:-1]
if len(blog2014_files)>200 :
    blog2014_files = np.random.choice(blog2014_files[:-1], 200)
    print "blog2014_files",len(blog2014_files)
else:
    blog2014_files = blog2014_files[:-1]
if len(review2014_files)>200:
    review2014_files = np.random.choice(review2014_files[:-1], 200)
    print "review2014_files",len(review2014_files)
else:
    review2014_files = review2014_files[:-1]
if len(social2014_files)>200:
    social2014_files = np.random.choice(social2014_files[:-1], 200)
    print "review2014_files", len(social2014_files)
else:
    social2014_files = social2014_files[:-1]
if len(twitter2015_files)>200:
    twitter2015_files = np.random.choice(twitter2015_files[:-1], 200)
    print "twitter2015_files", len(twitter2015_files)
else:
    twitter2015_files = twitter2015_files[:-1]
if len(twitter2016_files)>200:
    twitter2016_files = np.random.choice(twitter2016_files[:-1], 200)
    print "twitter2016_files", len(twitter2016_files)
else:
    twitter2016_files = twitter2016_files[:-1]

blog2013_files 200
review2014_files 200
review2014_files 200
twitter2016_files 200


# Blog 2013

In [5]:
count = 0
blog2013_df = pd.DataFrame(columns=('file', 'text', 'gender', 'age'))
for file in blog2013_files:
    #print file
    author = {}
    author['file'] = file
    with open(blog2013_path+file,'r') as f:
        tree = ET.parse(f)
        root = tree.getroot()
        author['gender'] = root.attrib['gender']
        author['age'] = root.attrib['age_group']
        #print gender, age
        raw_text = []
        for node in tree.iter('conversation'):
            #print node.tag, node.attrib
            #print node.text
            try:
                raw_text.append(BS(node.text, 'html.parser').get_text().strip())
            except:
                continue
        author['text'] = '\n'.join(raw_text)
    author_series = pd.Series(author)
    author_series.name = author['file']
#     author_series =  pd.Series(dict(zip(['file', 'text', 'gender', 'age'], [author['file'],author['text'],author['gender'],author['age']])))
    author_series.name = author['file']
    blog2013_df = blog2013_df.append(author_series)
    count += 1
    if count%50 == 0:
        print count," Records processsed"
blog2013_df.head()        

50  Records processsed
100  Records processsed
150  Records processsed
200  Records processsed


,file,text,gender,age
4040cd1f2cdc51a847dfadfed9fb6e24_en_30s_female.xml,4040cd1f2cdc51a847dfadfed9fb6e24_en_30s_female...,Give and it wil come back to u.\nbe comfortabl...,female,30s
84202b89d4b6bf136998efd49e3c976b_en_30s_female.xml,84202b89d4b6bf136998efd49e3c976b_en_30s_female...,Any affordable resort in London that is inside...,female,30s
3e6dafa50b26745648707ec61e8278d0_en_30s_male.xml,3e6dafa50b26745648707ec61e8278d0_en_30s_male.xml,Canines are among the most common sorts of pet...,male,30s
9ba8b4c876ae609ec20d518de169dcfb_en_20s_male.xml,9ba8b4c876ae609ec20d518de169dcfb_en_20s_male.xml,It is important to report your injury as soon ...,male,20s
19b9cdf91fa54461c2c344831f3da2be_en_30s_male.xml,19b9cdf91fa54461c2c344831f3da2be_en_30s_male.xml,Decorating your home used to be really simple-...,male,30s


# Review 2014

In [6]:
review2014_path = "G:\\author profiling\\pan14-author-profiling-training-corpus-english-reviews-2014-04-16\\"
truth_file = os.listdir(blog2014_path)[-1]
reviews_truth_df = pd.read_csv(review2014_path+truth_file, sep=":::", header=None, names=('file','gender','age'))
# print truth_df.columns.values
reviews_truth_df = reviews_truth_df.set_index('file')
review_df = pd.DataFrame(columns=('file', 'text', 'gender', 'age'))
count = 0
for file in review2014_files:
    author = {}
    author['file'] = file[:-4]
    #print file
    with open(review2014_path+file) as f:
        tree = ET.parse(f)
        raw_text = []        
        for node in tree.iter('document'):
            raw_text.append(BS(node.text, 'html.parser').get_text().strip())
        author['text'] = '\n'.join(raw_text)
        author['gender'] = reviews_truth_df.loc[author['file']]['gender']
        #print author['gender']
        author['age'] = reviews_truth_df.loc[author['file']]['age']
    author_series = pd.Series(author)
    author_series.name = author['file']
#     author_series =  pd.Series(dict(zip(['file', 'text', 'gender', 'age'], [author['file'],author['text'],author['gender'],author['age']])))
    author_series.name = author['file']
    review_df = review_df.append(author_series)
    count += 1
    if count%50 == 0:
        print count," records processed"
review_df.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


50  records processed
100  records processed
150  records processed
200  records processed


,file,text,gender,age
a11889bb92d6969286194525e90bfae1,a11889bb92d6969286194525e90bfae1,Excellent Hotel for the right traveller! I rea...,FEMALE,35-49
ccec4a7f232815502c8d8474bf76b95a,ccec4a7f232815502c8d8474bf76b95a,Loved the Mayfair Hotel/Location We returned l...,FEMALE,65-xx
3a089701a1d2b65589b209fff22bc512,3a089701a1d2b65589b209fff22bc512,Wonderful hotel! Can't beat Iberostar when it ...,FEMALE,25-34
2ccbc2d25d9c2095fbd8f85954954c12,2ccbc2d25d9c2095fbd8f85954954c12,"For the most part,wonderful! We enjoyed Culebr...",MALE,50-64
4d392c7a6e253558c4068fa3ca121d76,4d392c7a6e253558c4068fa3ca121d76,I just don't get this hotel - So terrible I ju...,MALE,35-49


# Social Media

In [7]:
#social2014_path = "G:\\author profiling\\pan14-author-profiling-training-corpus-english-socialmedia-2014-04-16\\"
socialmedia_truth_df = pd.read_csv(social2014_path+os.listdir(social2014_path)[-1], sep=":::", header=None, names=('file','gender','age'))
socialmedia_truth_df = socialmedia_truth_df.set_index('file')
social_df = pd.DataFrame(columns=('file', 'text', 'gender', 'age'))
count = 0
for file in social2014_files:
    author = {}
    author['file'] = file[:-4]
    #print file
    with open(social2014_path+file) as f:
        tree = ET.parse(f)
        raw_text = []        
        for node in tree.iter('document'):
            raw_text.append(BS(BS(node.text, 'html.parser').get_text().strip(),'html.parser').get_text())            
        author['text'] = '\n'.join(raw_text)
        author['gender'] = socialmedia_truth_df.loc[author['file']]['gender']
        #print author['gender']
        author['age'] = socialmedia_truth_df.loc[author['file']]['age']
    author_series = pd.Series(author)
    author_series.name = author['file']
    social_df = social_df.append(author_series)
    count += 1
    if count%50 == 0:
        print count, " records processed"
social_df.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


50  records processed


C:\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.powerofpositivity.info/http://www.powerofpositivity.info/
http://www.powerofpositivity.info/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


100  records processed
150  records processed
200  records processed


,file,text,gender,age
429653a8ee0a1220cb5f0fa67c2705f,429653a8ee0a1220cb5f0fa67c2705f,iphone repair TorontoIf you need to have a new...,FEMALE,35-49
e479736e602c1fc8bcc969b53ee37763,e479736e602c1fc8bcc969b53ee37763,On the internet Travel Portals - Generating Yo...,MALE,18-24
78dc1c79a97ad8772cc5d77e9fff676b,78dc1c79a97ad8772cc5d77e9fff676b,Starbucks to drop beetle juice from the menuSt...,MALE,25-34
3bb5355eac980b2c610dd5525d33e93c,3bb5355eac980b2c610dd5525d33e93c,homeopathy system of medicine explainedAs outl...,FEMALE,18-24
349bd6c48993d6b6d5e88b91003baefe,349bd6c48993d6b6d5e88b91003baefe,Sexy test> > This is the SEXY test.> > Post th...,MALE,18-24


# Twitter 2015

In [14]:
twitter2015_truth_df = pd.read_csv(twitter2015_path+os.listdir(twitter2015_path)[-1], sep=":::", header=None, names=('file','gender','age','p1','p2','p3','p4','p5'))
twitter2015_truth_df = twitter2015_truth_df.set_index('file')
twitter2015_df = pd.DataFrame(columns=('file', 'text', 'gender', 'age'))
count = 0
for file in twitter2015_files:
    author = {}
    author['file'] = file[:-4]
    with open(twitter2015_path+file) as f:
        tree = ET.parse(f)
        raw_text = []        
        for node in tree.iter('document'):
            try:
                raw_text.append(node.text)
            except:
                continue
            #print raw_text.append(BS(BS(node.text, 'html.parser').get_text().strip(),'html.parser').get_text())            
        author['text'] = '\n'.join(raw_text)
       b author['gender'] = twitter2015_truth_df.loc[author['file']]['gender']
        #print author['gender']
        author['age'] = twitter2015_truth_df.loc[author['file']]['age']
    #break
    author_series = pd.Series(author)
    author_series.name = author['file']
    twitter2015_df = twitter2015_df.append(author_series)
    count += 1
    if count%50 == 0:
        print count, " records processed"
twitter2015_df.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


50  records processed
100  records processed
150  records processed


,file,text,gender,age
02ae95de-7ee3-453a-978d-25d28b3f1a88,02ae95de-7ee3-453a-978d-25d28b3f1a88,Things I want for my business cards but are to...,M,25-34
03f72f70-7a00-4dbb-93b7-6c7f65954fc5,03f72f70-7a00-4dbb-93b7-6c7f65954fc5,"""If you fight for a cause then fight for a cau...",M,25-34
0aa7bace-924c-40fb-a2e5-3c7012ede244,0aa7bace-924c-40fb-a2e5-3c7012ede244,@username that tweet was mainly for you ;-) ch...,M,35-49
0b66092a-440e-4755-a624-759a580a1c70,0b66092a-440e-4755-a624-759a580a1c70,@username 'red' and 'blue' and then you wait t...,F,25-34
1094e809-1306-4cd1-8ac1-ebfdf7c872b8,1094e809-1306-4cd1-8ac1-ebfdf7c872b8,I sit with a boy in english he became my frien...,F,18-24


# Twitter 2016

In [29]:
twitter2016_truth_df = pd.read_csv(twitter2016_path+os.listdir(twitter2016_path)[-1], sep=":::", header=None, names=('file','gender','age'))
twitter2016_truth_df = twitter2016_truth_df.set_index('file')
twitter2016_df = pd.DataFrame(columns=('file', 'text', 'gender', 'age'))
count = 0
# print twitter2016_files
for file in twitter2016_files:
    author = {}
    author['file'] = file[:-4]
#     print author['file']
    with open(twitter2016_path+file) as f:
        try:
            tree = ET.parse(f)
            raw_text = []        
            for node in tree.iter('document'):
                raw_text.append(BS(node.text, 'html.parser').get_text().strip())
        except:
            continue
        author['text'] = '\n'.join(raw_text)
        author['gender'] = twitter2016_truth_df.loc[author['file']]['gender']
#         print author['text']
        author['age'] = twitter2016_truth_df.loc[author['file']]['age']
#     break
    author_series = pd.Series(author)
    author_series.name = author['file']
    twitter2016_df = twitter2016_df.append(author_series)
    count += 1
    if count%50 == 0:
        print count, " records processed"
twitter2016_df.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


1e51e1f0ccf130be68b42fdaf201f40e
88d1f93972e3c4462b85bc2fa7a9eaae
3316270c2e911fc9ee491dbad386ed4e
815c739d16821a8df42594567ea83862
11a689d54c4af2394bc91b4003e527f3
b0d2501202d406fd07ab84db6f751543
491f51cc73a8136a66ecfa5349567d08
4e100982097898ed43731519079b2624
9aa58b658935d017e0827d1b42e75596
96f5135be8b958006946821b351a1d80
3316270c2e911fc9ee491dbad386ed4e
611d21bbf8ba18cfedb76104c117ff76
9120e3d36669937b08310ac9b2523550
a837091210532f5e37916bba89654b2d
066df9d2383e006d1bf69f0e27732988
15cf31e89178b2894ea38869d8c6c75e
05f97ea55282c4f81fce6fa5751a7ef4
f9aaf0e979eda888db209a8f457ce7d7
717be9f4b9f1232c9302e3e3eff2239c
c4aa5802bf8371620331a83d9de85951
153a084dc32a339d356619232f3dea59
c297b3ecf103e67237c8a42659a178e3
44be30b800f13facaed7cda823976bf1
871f9e9590b7102a0c5021ea30f388d2
62fce7e595e9d3b9255ba68bc0ca4f67
5f29914eb034d37c58a4c06e8eae0bc9
ae50c06b7772995439999108e8a94784
4687f84cb54bda94350da7f67e51a823
6f234b4a1744e80f260f5684433f8598
52d582497ed56183efaf715e3988b628
7770144c0f

C:\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://bit.ly/bGdt9" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://bit.ly/12YVa7" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.examiner.com/examiner/x-25094-LA-Equine-Policy-Examiner~y2009m12d29-Wild-horse-roundup-gathers-nationwide-protests" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and

47fe62a96d93fd6ba25efb25fe7e7069
fe3e3e680ba647137f5b47fa931072b3
a0a70d414e0a11076419e96dfb3c84f4
62fce7e595e9d3b9255ba68bc0ca4f67
8de2105c1c2af55efadd26c38d57a6d7
d865063bfe8c8dd59b4d542b98e028d1


C:\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.bodalgo.com/voices/yeni-alvarez/index_e.php" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.youtube.com/watch?v=_jUqe0bYars" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


...

C:\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://20somethingfinance.com/pay-off-student-loan-debt-or-save-for-retirement/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


c330c997feb912afba713bad6bba2dcb
6c712b47c8e9db4dd7f0f1725045f7bc
115e7b931be3d52d35cb8ce2da09a1d7
a28f8dd03948a47b1144ad5df6ccc483
36b2593435e1bed13eb138c1973c13ed
8de2105c1c2af55efadd26c38d57a6d7
dd066409a86e37f7ba41ed32fad5fed9
6d5873bedcf09d9753c3c908ddf452af
ca59762fccbeb8a73f349c99a256a220
9354e4ff65d876fbf0bc17ac650c7054
f18213b41bb7311717493a931dc32296
b4b03d72bcf52773ec6b7c0a91ccfe1e
75470794aa1249218c8d5699018560ab
f677a164b16096fc2eb3af86694da8ac
c14f5f6eb08d47b72bc8dae3187aad75
361a5b6f327d74344f0f8c4cd7c27970
fd5b4c852ce49baef7a888198853917c
150  records processed
f5abf96f244c876d17ecf69863cb0abb
61244154e362cece7e2d3c1139948bde
d3fbad0a96845119d5dbd0e772c9fe53
ac9bec22ac3b5c33abd7299845283f9f
3de105c51ab70cbf61df2b36fb5a0fd4
3373639b1d50d755c03c02cd9b93b720
9c66f13c8038d9cec5fd2a754d959c2f
b00d4a7efad641297d13e214a80a00c0


C:\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://bit.ly/1Jbam" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


40b8af7721fd105d0f5ecb5dad74a991


,file,text,gender,age
1e51e1f0ccf130be68b42fdaf201f40e,1e51e1f0ccf130be68b42fdaf201f40e,More great coverage of the #divestment campaig...,MALE,25-34
88d1f93972e3c4462b85bc2fa7a9eaae,88d1f93972e3c4462b85bc2fa7a9eaae,"@TomNixonSpeaks, sign up for Klout and see how...",MALE,50-64
3316270c2e911fc9ee491dbad386ed4e,3316270c2e911fc9ee491dbad386ed4e,Problemática del Analista es a Cuál Jugador Re...,MALE,65-xx
815c739d16821a8df42594567ea83862,815c739d16821a8df42594567ea83862,Nueva imagen de Batman vs Deathstroke en el ví...,MALE,35-49
11a689d54c4af2394bc91b4003e527f3,11a689d54c4af2394bc91b4003e527f3,"If you've been involved in an auto accident, y...",MALE,35-49
